In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
!pip install --upgrade xlrd #colab bug verson xlrd
!pip install geopandas
!pip install pandas fiona shapely pyproj rtree
!pip install contextily
!pip install pygeos
!pip install topojson
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
#!pip install requests py7zr geopandas openpyxl tqdm s3fs PyYAML xlrd
#!pip install git+https://github.com/inseefrlab/cartiflette@80b8a5a28371feb6df31d55bcc2617948a5f9b1a

In [2]:
#Importation de la base détaillant les origines de la pollution
data_pollution = pd.read_csv("https://koumoul.com/data-fair/api/v1/datasets/igt-pouvoir-de-rechauffement-global/convert", encoding = 'utf-8')

In [3]:
#Dictionnaire associant les numéros de département à ceux des régions

correspondance_reg = {
    84: ['01', '03', '07', '15', '26', '38', '42', '43', '63', '69', '73', '74'],  # Auvergne-Rhône-Alpes
    27: ['21', '25', '39', '58', '70', '71', '89', '90'],  # Bourgogne-Franche-Comté
    53: ['22', '29', '35', '56'],  # Bretagne
    28: ['18', '28', '36', '37', '41', '45'],  # Centre-Val de Loire
    94: ['2A', '2B'],  # Corse
    44: ['08', '10', '51', '52', '54', '55', '57', '67', '68', '88'],  # Grand Est
    32: ['02', '59', '60', '62', '80'],  # Hauts-de-France
    11: ['75', '77', '78', '91', '92', '93', '94', '95'],  # Île-de-France
    76: ['14', '27', '50', '61', '76'],  # Normandie
    75: ['16', '17', '19', '23', '24', '33', '40', '47', '64', '79', '86', '87'],  # Nouvelle-Aquitaine
    24: ['09', '11', '12', '30', '31', '32', '34', '46', '48', '65', '66', '81', '82'],  # Occitanie
    52: ['44', '49', '53', '72', '85'],  # Pays de la Loire
    93: ['04', '05', '06', '13', '83', '84']  # Provence-Alpes-Côte d'Azur
}

correspondance_reg = {dep: region for region, deps in correspondance_reg.items() for dep in deps}

In [4]:
#Création de la variable département à partir du code commune de l'INSEE
data_pollution['DEP'] = data_pollution['INSEE commune'].str[:2]

In [12]:
#Création de la variable région à partir de celle du département
size = len(data_pollution['DEP'])
data_pollution['REG'] = pd.Series([0]*size)

for i in range(size) :
    data_pollution.loc[i, 'REG'] = correspondance_reg.get(data_pollution['DEP'][i],"inconnu")

In [13]:
data_pollution

,INSEE commune,Commune,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire,DEP,REG
0,01001,L'ABERGEMENT-CLEMENCIAT,3711.425991,NaN,NaN,432.751835,101.430476,2.354558,6.911213,309.358195,793.156501,367.036172,01,84
1,01002,L'ABERGEMENT-DE-VAREY,475.330205,NaN,NaN,140.741660,140.675439,2.354558,6.911213,104.866444,348.997893,112.934207,01,84
2,01004,AMBERIEU-EN-BUGEY,499.043526,212.577908,NaN,10313.446515,5314.314445,998.332482,2930.354461,16616.822534,15642.420313,10732.376934,01,84
3,01005,AMBERIEUX-EN-DOMBES,1859.160954,NaN,NaN,1144.429311,216.217508,94.182310,276.448534,663.683146,1756.341319,782.404357,01,84
4,01006,AMBLEON,448.966808,NaN,NaN,77.033834,48.401549,NaN,NaN,43.714019,398.786800,51.681756,01,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35793,95676,VILLERS-EN-ARTHIES,1628.065094,NaN,NaN,165.045396,65.063617,11.772789,34.556067,176.098160,309.627908,235.439109,95,11
35794,95678,VILLIERS-ADAM,698.630772,NaN,NaN,1331.126598,111.480954,2.354558,6.911213,1395.529811,18759.370071,403.404815,95,11
35795,95680,VILLIERS-LE-BEL,107.564967,NaN,NaN,8367.174532,225.622903,534.484607,1568.845431,22613.830247,12217.122402,13849.512001,95,11
35796,95682,VILLIERS-LE-SEC,1090.890170,NaN,NaN,326.748418,108.969749,2.354558,6.911213,67.235487,4663.232127,85.657725,95,11


In [17]:
REG_DEP_na = data_pollution[(data_pollution['DEP'].isna()) | (data_pollution['REG'].isna())]
REG_DEP_na.shape
#L'avantage par rapport au code précédent est qu'il n'y a plus de données perdues 
#ce qui n'était pas le cas avec le merge avec base_geo_pour_merge 

(0, 14)

In [19]:
#Somme des niveaux de pollution par département
tentative_groupes_1 = data_pollution.drop(['Commune','INSEE commune', 'REG'], axis = 1).groupby('DEP').sum()
tentative_groupes_1

,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire
DEP,,,,,,,,,,
01,8.075850e+05,15647.945725,2.581109e+02,7.119048e+05,275415.025873,113596.600977,706549.858520,5.522627e+05,1.635350e+06,418646.522003
02,1.276261e+06,58834.836998,2.521642e+03,6.174937e+05,180245.578283,60366.151368,737319.268428,6.388608e+05,1.386403e+06,325014.134252
03,1.949985e+06,19686.272920,5.451597e+02,5.659226e+05,111219.488315,102975.024821,457513.404309,4.457248e+05,1.164762e+06,224487.970942
04,3.632657e+05,5851.619941,0.000000e+00,1.160564e+05,50541.206026,12184.836311,60899.229209,1.168361e+05,3.905682e+05,98228.256513
05,3.103814e+05,6216.001726,0.000000e+00,8.433816e+04,22268.075401,5455.510286,16013.281334,1.224274e+05,3.458587e+05,77885.624538
...,...,...,...,...,...,...,...,...,...,...
91,1.565447e+05,839544.846964,1.257594e+06,7.285136e+05,293317.174006,105597.205576,368638.793346,1.280296e+06,2.073377e+06,757884.411113
92,9.140818e+01,12340.794839,2.101194e+02,1.067889e+06,264497.880711,242842.018012,706597.424067,1.466794e+06,1.198420e+06,836013.153414
93,2.018471e+03,59617.086124,1.101400e+06,7.259516e+05,252166.943778,102837.663903,433216.360990,1.316452e+06,1.396911e+06,863017.791777


In [20]:
#Somme des niveaux de pollution par région
tentative_groupes_2 = data_pollution.drop(['Commune','INSEE commune', 'DEP'], axis = 1).groupby('REG').sum()
tentative_groupes_2

,Agriculture,Autres transports,Autres transports international,CO2 biomasse hors-total,Déchets,Energie,Industrie hors-énergie,Résidentiel,Routier,Tertiaire
REG,,,,,,,,,,
11,1.092568e+06,1.677456e+06,1.258802e+07,6.768090e+06,1.969944e+06,1.847798e+06,5.110941e+06,1.153692e+07,1.381316e+07,6.973132e+06
24,9.216468e+06,9.403150e+05,7.127516e+05,5.085909e+06,1.148631e+06,5.235770e+05,3.855530e+06,4.601612e+06,1.270235e+07,3.415642e+06
27,8.058068e+06,2.734776e+05,1.158992e+04,4.371869e+06,5.557502e+05,4.486914e+05,2.756607e+06,3.432179e+06,7.942005e+06,1.814003e+06
28,5.429369e+06,7.471300e+04,1.541066e+04,3.215651e+06,4.653129e+05,3.900968e+05,2.351066e+06,2.841666e+06,6.865600e+06,1.698426e+06
32,6.208521e+06,3.312563e+05,1.134901e+06,5.419011e+06,1.394532e+06,2.955446e+06,1.594452e+07,7.333624e+06,1.118120e+07,3.713517e+06
44,8.733498e+06,5.292856e+05,7.605698e+05,7.295111e+06,1.227595e+06,1.327028e+06,1.203603e+07,7.336476e+06,1.252523e+07,3.692463e+06
52,8.572066e+06,2.553621e+05,3.820464e+05,3.730637e+06,7.306768e+05,1.976390e+06,3.471932e+06,3.438540e+06,7.562122e+06,2.352564e+06
53,7.663104e+06,2.846105e+05,8.055797e+04,3.331220e+06,6.508873e+05,3.985401e+05,1.590004e+06,2.892887e+06,6.517828e+06,2.085653e+06
75,1.298515e+07,4.415826e+05,8.782023e+05,7.143234e+06,1.427269e+06,6.529124e+05,5.175242e+06,5.802500e+06,1.397049e+07,3.653666e+06


In [29]:
set_dep_littoral = (59, 62, 80, 76, 27, 14, 50, 35, 
                      22, 29, 56, 44, 85, 17, 33, 40, 
                    64, 66, 11, 34,
                    30, 13, 83, 0o6, '2A', '2B')
print(set_dep_littoral)

(59, 62, 80, 76, 27, 14, 50, 35, 22, 29, 56, 44, 85, 17, 33, 40, 64, 66, 11, 34, 30, 13, 83, 6, '2A', '2B')


In [ ]:
# Importation de la base en local pour faire tourner le code main
data_pollution.to_csv('base_pollution.csv', sep=',', index=False, encoding='utf-8')